In [1]:
import numpy as np

In [5]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jamiewelsh2/rap-lyrics")

print("Path to dataset files:", path)


# Load the dataset
df = pd.read_csv(path + "/updated_rappers.csv")
display(df.head())

Path to dataset files: C:\Users\Nima\.cache\kagglehub\datasets\jamiewelsh2\rap-lyrics\versions\1


,Unnamed: 0,artist,song,lyric,next lyric
0,0,Fetty Wap,Trap Queen,rgf productions,remy boyz yahah
1,0,Fetty Wap,Trap Queen,remy boyz yahah,1738 ayy
2,0,Fetty Wap,Trap Queen,1738 ayy,im like hey whats up hello
3,0,Fetty Wap,Trap Queen,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,0,Fetty Wap,Trap Queen,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [6]:
# Load data
def load_data(path):
    df = pd.read_csv(path + "/updated_rappers.csv")
    df = df[['lyric', 'next lyric']].dropna()
    return df

df = load_data(path)
display(df.head())

,lyric,next lyric
0,rgf productions,remy boyz yahah
1,remy boyz yahah,1738 ayy
2,1738 ayy,im like hey whats up hello
3,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [7]:
# Preprocess data
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Special characters weghalen
    text = re.sub(r'\s+', ' ', text).strip()  # Extra whitespace weghalen
    return text


In [8]:
# toepassen van clean_text op de kolommen lyric en next lyric
def preprocess_lyrics(df):
    df['lyric'] = df['lyric'].apply(clean_text)
    df['next lyric'] = df['next lyric'].apply(clean_text)
    return df



In [9]:
# Tokenize data
from collections import Counter
def build_vocab(df, vocab_size=5000):
    words = ' '.join(df['lyric']).split()
    word_counts = Counter(words)
    most_common = word_counts.most_common(vocab_size - 1)
    vocab = {word: idx+1 for idx, (word, _) in enumerate(most_common)}  
    vocab['<UNK>'] = vocab_size  # Unknown words
    return vocab


def text_to_sequence(text, vocab):
    return [vocab.get(word, vocab['<UNK>']) for word in text.split()]

# How do I make train and test set the same size using bag of words. (n.d.).
# Stack Overflow. https://stackoverflow.com/questions/67494914/how-do-i-make-train-and-test-set-the-same-size-using-bag-of-words

In [10]:
# Data naar numerieke waarden omzetten
def prepare_data(filepath, vocab_size=5000):
    df = load_data(filepath)
    df = preprocess_lyrics(df)
    vocab = build_vocab(df, vocab_size)
    df['lyric_seq'] = df['lyric'].apply(lambda x: text_to_sequence(x, vocab))
    df['next_lyric_seq'] = df['next lyric'].apply(lambda x: text_to_sequence(x, vocab))
    return df, vocab

In [11]:
df, vocab = prepare_data(path)

In [12]:
display(df.head())

,lyric,next lyric,lyric_seq,next_lyric_seq
0,rgf productions,remy boyz yahah,"[5000, 5000]","[2462, 3022, 5000]"
1,remy boyz yahah,ayy,"[2462, 3022, 5000]",[321]
2,ayy,im like hey whats up hello,[321],"[11, 14, 227, 193, 18, 929]"
3,im like hey whats up hello,seen yo pretty ass soon as you came in the door,"[11, 14, 227, 193, 18, 929]","[254, 98, 568, 124, 622, 84, 3, 199, 9, 1, 418]"
4,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll,"[254, 98, 568, 124, 622, 84, 3, 199, 9, 1, 418]","[2, 36, 72, 757, 21, 4, 1583, 19, 122, 6, 255]"


Hier is een uitgebreide, gestructureerde uitleg van de **Mamba-class**, inclusief duidelijke referenties naar de paper. Je kunt dit direct in je Markdown-document plaatsen.  

---

## **Implementatie en uitleg van het Mamba-model**  

De **Mamba-architectuur** is een nieuw type sequentiemodel dat gebruik maakt van **State Space Models (SSMs)**, in plaats van zelf-attentie zoals in Transformers. In tegenstelling tot klassieke SSMs, die een vast geheugen hebben, introduceert Mamba een **selectief mechanisme** dat het mogelijk maakt om irrelevante informatie te filteren en belangrijke tokens vast te houden. Dit maakt het model efficiënter en beter in het verwerken van lange sequenties.  

Onze implementatie is gebaseerd op de principes uit de paper *Mamba: Linear-Time Sequence Modeling with Selective State Spaces* (Gu & Dao, 2024) en bouwt stap voor stap een eenvoudige versie van het model op.  

---

### **1. De Basis: State Space Models (SSMs)**  

In de paper wordt uitgelegd dat **State Space Models (SSMs)** werken met een verborgen toestand (\( h_t \)) die bij elke nieuwe token wordt bijgewerkt volgens de volgende formule:  

\[
h_t = A h_{t-1} + B x_t
\]

en de uiteindelijke output wordt berekend als:

\[
y_t = C h_t
\]

Dit betekent dat de toestand van het model op elk moment afhangt van de vorige toestand (\( h_{t-1} \)) en de huidige input (\( x_t \)). Dit concept komt uit de klassieke **lineaire recursieve netwerken** en wordt gebruikt om lange-afstandsafhankelijkheden in tekst te modelleren.  

**Referentie:** Zie sectie *State Space Models* (pagina 3) waar deze basisformules worden geïntroduceerd:  

*"S4 models are defined with four parameters (Δ, A, B, C), which define a sequence-to-sequence transformation in two stages."*  

Om dit in onze code om te zetten, gebruiken we een eenvoudige implementatie waarbij een verborgen toestand (\( h_t \)) wordt bijgehouden en geüpdatet bij elke nieuwe token:  

```python
self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
```

Hierbij:
- **\( W \)** is de matrix die de toestand bijwerkt op basis van de vorige toestand.
- **\( U \)** transformeert de invoer naar een vector die kan worden toegevoegd aan de verborgen toestand.
- **\( b \)** is een bias-term.
- **\( \tanh \)** zorgt ervoor dat de waarden binnen een bepaald bereik blijven en voorkomt dat de toestand explodeert.  

---

### **2. Selectieve Informatieopslag: De Verbetering van Mamba**  

Een van de kernproblemen van eerdere SSMs was dat ze **tijd-invariant** waren: de overgangsmatrix (\( A \)) veranderde niet afhankelijk van de invoer. Dit betekende dat ze **geen content-afhankelijke beslissingen konden nemen**, zoals wanneer een woord belangrijk is om te onthouden.  

Mamba lost dit op door de **SSM-parameters input-afhankelijk te maken**. Dit betekent dat de manier waarop het model informatie doorgeeft, varieert afhankelijk van de invoer die het krijgt. Hierdoor kan Mamba:  
- Relevante tokens opslaan en irrelevante negeren.  
- Informatie veel efficiënter doorgeven.  

**Referentie:** Zie sectie *Selection Mechanism* (pagina 5), waar wordt uitgelegd hoe Mamba een selectiemechanisme toevoegt aan de klassieke SSM-structuur:  

*"Building on intuition based on important synthetic tasks such as selective copy and induction heads, we design a simple selection mechanism by parameterizing the SSM parameters based on the input."*  

Onze implementatie vertaalt dit concept als volgt:  

```python
for token in input_seq:
    x = np.zeros(self.vocab_size)
    x[token] = 1  # One-hot encoding van de input
    self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
```

Hier zorgt de **one-hot encoding** ervoor dat elk woord wordt omgezet in een vector, en de overgangsformule past zich dynamisch aan op basis van deze invoer.

---

### **3. Autoregressieve Generatie**  

Omdat Mamba ontworpen is als een **autoregressief model**, kan het worden gebruikt om tekst te genereren. Dit betekent dat we het model een beginzin geven en het vervolgens **stap voor stap nieuwe tokens laat voorspellen** op basis van de verborgen toestand.  

De paper beschrijft dat Mamba **geen cache van eerdere tokens nodig heeft**, in tegenstelling tot Transformers. Dit maakt het model veel efficiënter in inference.  

**Referentie:** Zie sectie *Inference and Scaling* (pagina 2):  

*"Unrolling the model autoregressively during inference requires only constant time per step since it does not require a cache of previous elements."*  

In onze implementatie gebeurt dat zo:  

```python
def generate(self, start_seq, length=10):
    generated = list(start_seq)
    for _ in range(length):
        state = self.forward(generated[-1:])
        next_token = np.argmax(state)  # Kies de meest waarschijnlijke volgende token
        generated.append(next_token)
    return generated
```

Hierbij:
- **We starten met een beginzin** (`start_seq`).
- **Elke nieuwe token wordt gegenereerd door het model** op basis van de vorige token.
- **De output wordt gegenereerd met `np.argmax(state)`**, wat de meest waarschijnlijke token selecteert.

Dit betekent dat als we bijvoorbeeld de input `"ayy"` geven, het model mogelijk `"im like hey whats up"` genereert als de volgende woorden.








In [13]:
# Mamba model implementeren
class Mamba:
    def __init__(self, vocab_size, hidden_size=128):
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.W = np.random.randn(hidden_size, hidden_size) * 0.1
        self.U = np.random.randn(hidden_size, vocab_size) * 0.1
        self.b = np.zeros(hidden_size)
        self.hidden_state = np.zeros(hidden_size)
    
    def forward(self, input_seq):
        for token in input_seq:
            x = np.zeros(self.vocab_size)
            x[token] = 1  # One-hot encoding
            self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
        return self.hidden_state
    
    def generate(self, start_seq, length=10):
        generated = list(start_seq)
        for _ in range(length):
            state = self.forward(generated[-1:])
            next_token = np.argmax(state)  # Simplified selection
            generated.append(next_token)
        return generated
    


In [14]:
# example usage
model = Mamba(vocab_size=len(vocab))
generated_seq = model.generate([vocab['nigga']], length=30)

# Convert sequence back to text
generated_text = ' '.join([list(vocab.keys())[idx] for idx in generated_seq])
print(generated_text)


just go need then bitch me so ill yeah when these baby never here never feel baby my just my hit take never still baby never me have still you shit


In [22]:
import numpy as np
import random
import pandas as pd

lyrics = df['lyric'].dropna().tolist()[:20000]  # Gebruik alleen de eerste 10.000 rijen voor test

# Eenvoudige tokenizer (woord naar ID)
def build_vocab(lyrics):
    vocab = {word: idx for idx, word in enumerate(set(" ".join(lyrics).split()))}
    vocab["<UNK>"] = len(vocab)  # Onbekende woorden
    return vocab

# Omgekeerde vocab (ID naar woord)
def build_reverse_vocab(vocab):
    return {idx: word for word, idx in vocab.items()}

# Converteer tekst naar ID's
def text_to_sequence(text, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in text.split()]

# Converteer ID's terug naar tekst
def sequence_to_text(sequence, reverse_vocab):
    return " ".join([reverse_vocab[idx] for idx in sequence])

# Geoptimaliseerd generatief Mamba-achtig model
class GenerativeMamba:
    def __init__(self, vocab_size, hidden_size=128, learning_rate=0.01, batch_size=4, temperature=1.0):
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.temperature = temperature
        
        # Gewichtsmatrices voor vectorverwerking
        self.W = np.random.randn(hidden_size, vocab_size) * 0.01  # Input -> Hidden
        self.U = np.random.randn(hidden_size, hidden_size) * 0.01  # Hidden -> Hidden
        self.V = np.random.randn(vocab_size, hidden_size) * 0.01  # Hidden -> Output
        
        # Initialiseer hidden state
        self.h = np.zeros((self.batch_size, hidden_size))
    
    def step(self, X_batch):
        H = np.tanh(np.dot(X_batch, self.W.T) + np.dot(self.h, self.U.T))  # State update voor batch
        output = np.dot(H, self.V.T)  # Output berekenen
        self.h = H  # Hidden state update
        return output
    
    def train(self, sequences, epochs=50):
        num_samples = len(sequences)
        
        for epoch in range(epochs):
            total_loss = 0
            
            for i in range(0, num_samples, self.batch_size):
                batch = sequences[i:i + self.batch_size]
                batch_size_actual = len(batch)
                X_batch = np.zeros((batch_size_actual, len(vocab)))
                Y_batch = np.zeros((batch_size_actual, len(vocab)))
                
                for j, seq in enumerate(batch):
                    if len(seq) > 1:
                        X_batch[j, seq[:-1]] = 1  # One-hot input
                        Y_batch[j, seq[1:]] = 1  # Verwachte output
                
                logits = self.step(X_batch)
                logits = logits - np.max(logits, axis=1, keepdims=True)  # **Numerieke stabiliteit fix**
                probs = np.exp(logits / self.temperature)
                probs /= np.sum(probs, axis=1, keepdims=True)  # **Zorg dat som = 1**
                
                if np.any(np.isnan(probs)) or np.any(np.isinf(probs)):
                    print("Waarschuwing: ongeldige waarschijnlijkheden gedetecteerd, standaard fallback gebruikt.")
                    probs = np.nan_to_num(probs, nan=1e-9)  # Vervang NaN's met kleine waarde
                
                loss = -np.sum(Y_batch * np.log(probs + 1e-9)) / batch_size_actual  # Gemiddelde loss
                total_loss += loss
                
                # Backpropagation
                dL_dV = np.dot((probs - Y_batch).T, self.h)  # Correcte vorm
                dL_dh = np.dot((probs - Y_batch), self.V) * (1 - self.h ** 2)
                dL_dW = np.dot(dL_dh.T, X_batch)
                dL_dU = np.dot(dL_dh.T, self.h)
                
                # Update gewichten
                self.V -= self.learning_rate * dL_dV
                self.W -= self.learning_rate * dL_dW
                self.U -= self.learning_rate * dL_dU
            
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")
    
    def generate(self, start_seq, vocab, reverse_vocab, length=10):
        seq = text_to_sequence(start_seq, vocab)
        generated = seq[:]
        
        for _ in range(length):
            x = np.zeros(len(vocab))
            x[seq[-1]] = 1  # One-hot encoding
            
            logits = self.step(x.reshape(1, -1))[0]
            logits = logits - np.max(logits)  # **Numerieke stabiliteit fix**
            probs = np.exp(logits / self.temperature)
            probs /= np.sum(probs)  # **Softmax correctie**
            
            if np.any(np.isnan(probs)) or np.any(np.isinf(probs)):
                print("Waarschuwing: ongeldige waarschijnlijkheden gedetecteerd, fallback naar argmax.")
                next_idx = np.argmax(logits)  # Gebruik argmax als fallback
            else:
                next_idx = np.random.choice(len(vocab), p=probs)  # **Correcte sampling**
                
            generated.append(next_idx)
            seq.append(next_idx)
            
        return sequence_to_text(generated, reverse_vocab)

# Vocab maken
vocab = build_vocab(lyrics)
reverse_vocab = build_reverse_vocab(vocab)

# Convert dataset naar sequenties
sequences = [text_to_sequence(line, vocab) for line in lyrics]

# Model maken en trainen
generator = GenerativeMamba(vocab_size=len(vocab), batch_size=4, temperature=0.8)
generator.train(sequences, epochs=40)



Epoch 1/40, Loss: 544053.5890
Epoch 2/40, Loss: 553453.7766
Epoch 3/40, Loss: 553493.4035
Epoch 4/40, Loss: 553521.6863
Epoch 5/40, Loss: 553540.9715
Epoch 6/40, Loss: 553553.1132
Epoch 7/40, Loss: 553559.4988
Epoch 8/40, Loss: 553562.1338
Epoch 9/40, Loss: 553562.8328
Epoch 10/40, Loss: 553562.9410
Epoch 11/40, Loss: 553562.9541
Epoch 12/40, Loss: 553562.9557
Epoch 13/40, Loss: 553562.9559
Epoch 14/40, Loss: 553562.9560
Epoch 15/40, Loss: 553562.9560
Epoch 16/40, Loss: 553562.9560
Epoch 17/40, Loss: 553562.9560
Epoch 18/40, Loss: 553562.9561
Epoch 19/40, Loss: 553562.9561
Epoch 20/40, Loss: 553562.9561
Epoch 21/40, Loss: 553562.9561
Epoch 22/40, Loss: 553562.9561
Epoch 23/40, Loss: 553562.9561
Epoch 24/40, Loss: 553562.9561
Epoch 25/40, Loss: 553562.9561
Epoch 26/40, Loss: 553562.9561
Epoch 27/40, Loss: 553562.9561
Epoch 28/40, Loss: 553562.9561
Epoch 29/40, Loss: 553562.9561
Epoch 30/40, Loss: 553562.9562
Epoch 31/40, Loss: 553562.9562
Epoch 32/40, Loss: 553562.9562
Epoch 33/40, Loss

In [23]:
# Genereer tekst
start = "she got"
print("Generated:", generator.generate(start, vocab, reverse_vocab, length=100))

Generated: she got to i on that i the me to the the the the in that the that on my the to the the the that you in like my the the you in the like the me in the my that a that like a you the the you to like the the the in my i the that the the i the my in it in the on my the my that to the im to my to a a it you to the my the i to that in i me a the it me i the the the
